# DreamJobber

**Tech Edition**

---

**Process**
1. Clean text
2. Bag of Words
3. LDA model (Latent Dirichlet allocation)
4. Fine tune LDA model
5. Define Topics from LDA model
6. Create df of document probabilities
6. Classification model

---

**Import Necessary Libraries**

In [35]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
from functions import *
import pickle

#lda model evaluatoin with coherence
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

from sklearn.neighbors import NearestNeighbors

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/admin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

---

**Load Data**

In [3]:
df_1 = pd.read_json('data/dice_jobs_1.json', lines=True)
df_2 = pd.read_json('data/dice_jobs_2.json', lines=True)
df_3 = pd.read_json('data/dice_jobs_3.json', lines=True)
df_4 = pd.read_json('data/dice_jobs_4.json', lines=True)
df_5 = pd.read_json('data/dice_jobs_5.json', lines=True)
df_6 = pd.read_json('data/dice_jobs_6.json', lines=True)

In [4]:
#concat into one df
df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6], ignore_index=True, sort=True)

In [5]:
df.head()

,job_description,job_title
0,NaN,UI Lead/Architect
1,NaN,Web Application Architect
2,NaN,Senior DataStage Developer
3,NaN,Hadoop Administrator
4,NaN,UX Visual Designer


In [6]:
#check for missing values
df.isna().sum()

job_description    6524
job_title          5484
dtype: int64

In [7]:
#looks like there are rows that have no job description
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27016 entries, 0 to 27015
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          21532 non-null object
dtypes: object(2)
memory usage: 422.2+ KB


In [8]:
#drop rows with no job descriptions
df = df.dropna()

In [9]:
#sanity check, looks good
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20492 entries, 9 to 27015
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          20492 non-null object
dtypes: object(2)
memory usage: 480.3+ KB


In [10]:
df.head()

,job_description,job_title
9,[2+ years of experience developing Java / J2EE...,Java Developer (Sign-On BONUS!)
10,"[Passion for technology and learning, a natura...","iOS Developer - Mobile Rate: Open, Duration: 1..."
17,[We enjoy approved IT vendor status with sever...,"EUC Engineer, Rate-Open, Duration: 18 Months"
18,[We enjoy approved IT vendor status with sever...,"Software Developer - RPG, Rate-Open, Duration:..."
19,[SME in Linux Operating system with Strong Vir...,Sr. Linux Consultant with Weblogic exp


In [11]:
#need to remove brackets from job_description
df['job_description'] = df['job_description'].map(remove_brackets)

In [12]:
#remove '\\n' and replace with ','
df['job_description'] = df['job_description'].map(lambda x: x.replace('\\n', ','))

In [13]:
#lowercase text before applying stopwords
df['job_description'] = df['job_description'].map(lambda x: x.lower())

In [14]:
#lowercase job_title text before cleaning
df['job_title'] = df['job_title'].map(lambda x: x.lower())

In [15]:
#Return first title from job_titles
df['job_title'] = df['job_title'].map(get_first_title)

In [16]:
#def remove_stop_words(text):
    
 #   """Remove stopwords from job titles"""
    
  #  jobtitle_stopwords = ['bonus', 
   #                       'duration',
    #                      'month',
     #                     'open',
      #                    'rate',
       #                   'sign-on',
        #                  'year']
    
    #result = []
    
    #for word in text:
    
     #   if word not in jobtitle_stopwords:
      #      result.append(text)
    #return result


In [17]:
#remove stop words from job titles
#df['job_title'] = df['job_title'].map(remove_stop_words)

In [18]:
df = df.reset_index(drop=True)
df.head()

,job_description,job_title
0,'2+ years of experience developing java / j2ee...,java developer
1,"'passion for technology and learning, a natura...",ios developer - mobile rate
2,'we enjoy approved it vendor status with sever...,euc engineer
3,'we enjoy approved it vendor status with sever...,software developer - rpg
4,'sme in linux operating system with strong vir...,sr. linux consultant with weblogic exp


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20492 entries, 0 to 20491
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          20492 non-null object
dtypes: object(2)
memory usage: 320.3+ KB


In [20]:
df = df.drop_duplicates()

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18709 entries, 0 to 20491
Data columns (total 2 columns):
job_description    18709 non-null object
job_title          18709 non-null object
dtypes: object(2)
memory usage: 438.5+ KB


---

## Text Cleaning

1. Tokenize
2. Remove words with fewer than 3 characters
3. Remove stop words
4. Normalize words (Lemmatize and Stem)

**Test the functions on one row of text**

In [22]:
stemmer = SnowballStemmer('english')

In [23]:
text_sample = df[df.index == 53].values[0][0]

print('original text: ')
words = []
for word in text_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized text: ')
print(preprocess(text_sample))

original text: 
["'we", 'enjoy', 'approved', 'it', 'vendor', 'status', 'with', 'several', 'leading', 'companies', 'in', 'the', 'pacific', 'northwest', 'and', 'continually', 'grow', 'our', 'list', 'of', 'approved', 'vendor', 'status', 'with', 'additional', "companies.',", "'we", 'are', 'an', 'ethical', 'company', 'with', 'integrity', 'and', 'a', 'dedication', 'to', 'delivering', 'high', 'results', 'for', 'our', "clients.',", "'we", 'are', 'fair', 'and', 'honest', 'in', 'all', 'of', 'our', 'business', 'dealings', 'with', 'our', 'consultants', 'and', "clients',", "'we", 'pay', 'top', 'scale', 'hourly', 'rates', 'based', 'on', 'your', 'credentials,', 'experience', 'and', 'market', 'demand', 'for', 'your', 'skill', "sets',", "'we", 'can', 'offer', 'other', 'related', 'benefits', 'as', 'needed', 'and', 'customized', 'to', 'your', "situation',", "'we", 'can', 'work', 'with', 'you', 'to', 'help', 'you', 'achieve', 'your', 'career', 'growth', 'and', "goals',", "'we", 'offer', 'immediate', 'h1b'

**Apply cleaning functions to job_description**

In [24]:
#apply function and display first 5 rows
processed_text = df['job_description'].map(preprocess)
processed_text[:10]

0                                  [develop, java, web]
1     [passion, technolog, learn, natur, curios, lov...
2     [enjoy, approv, vendor, status, lead, compani,...
3     [enjoy, approv, vendor, status, lead, compani,...
4     [sme, linux, oper, strong, virtual, knowledg, ...
5     [evalu, extract, transform, data, analyt, purp...
6     [techno, function, cpi, technic, expertis, int...
7                            [mainten, develop, legaci]
8     [enjoy, approv, vendor, status, lead, compani,...
10    [enjoy, approv, vendor, status, lead, compani,...
Name: job_description, dtype: object

---

## Bag of Words

In [25]:
#I'll use bag of words to extract features from text for use in modeling

In [26]:
dictionary = gensim.corpora.Dictionary(processed_text)

In [27]:
#check the length before I filter out the extremes
len(dictionary)

24557

In [28]:
dictionary.filter_extremes(no_below=25, no_above=0.5, keep_n=100000)

In [29]:
#check length after filtering out extremes
len(dictionary)

3753

In [30]:
#bow2doc: counts the number of occurrences of each distinct word, 
#converts the word to its integer word id and returns the result as a sparse vector

bow2doc_corpus = [dictionary.doc2bow(text) for text in processed_text]

---

## Find optimal number of topics

In [ ]:
#model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=bow2doc_corpus,
 #                                                       texts=processed_text, start=5, limit=40, step=5)

In [ ]:
#import matplotlib.pyplot as plt
#limit=40; start=5; step=5;
#x = range(start, limit, step)
#plt.plot(x, coherence_values)
#plt.xlabel("Number of Topics")
#plt.ylabel("Coherence score")
#plt.show()

---

## LDA model with Bag of Words

In [33]:
lda_model = gensim.models.LdaMulticore(bow2doc_corpus, 
                                       num_topics=10, 
                                       id2word=dictionary, 
                                       passes=50, 
                                       workers=4,
                                      chunksize=750)


In [34]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.035*"status" + 0.025*"receiv" + 0.024*"protect" + 0.022*"origin" + 0.022*"success" + 0.020*"engin" + 0.020*"qualifi" + 0.019*"regard" + 0.016*"group" + 0.015*"factor"
Topic: 1 
Words: 0.114*"data" + 0.051*"secur" + 0.016*"model" + 0.015*"system" + 0.015*"inform" + 0.014*"architect" + 0.013*"analyt" + 0.013*"analyst" + 0.011*"architectur" + 0.010*"analysi"
Topic: 2 
Words: 0.013*"client" + 0.010*"solut" + 0.010*"help" + 0.009*"compani" + 0.008*"consult" + 0.008*"industri" + 0.008*"learn" + 0.008*"profession" + 0.007*"build" + 0.007*"peopl"
Topic: 3 
Words: 0.033*"network" + 0.020*"system" + 0.014*"technic" + 0.013*"oper" + 0.012*"environ" + 0.011*"softwar" + 0.011*"hardwar" + 0.011*"troubleshoot" + 0.011*"window" + 0.011*"issu"
Topic: 4 
Words: 0.033*"market" + 0.021*"digit" + 0.018*"product" + 0.017*"creativ" + 0.014*"content" + 0.012*"web" + 0.010*"user" + 0.010*"creat" + 0.009*"media" + 0.009*"campaign"
Topic: 5 
Words: 0.024*"sql" + 0.022*"databas" + 0.013*"integr

**Pickled LDA model results**

In [ ]:
#pickle.dump(lda_model, open('pickled_models/lda_model.pkl', 'wb'))
pickled_lda = pickle.load(open('pickled_models/lda_model.pkl', 'rb'))

In [ ]:
for idx, topic in pickled_lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

---

**LDA model evaluation**

In [ ]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_text, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Compute Coherence Score using UMass
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_text, dictionary=dictionary, coherence="u_mass")
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
#!pip install pyLDAvis

In [ ]:
##visualize the topics in order to better label 
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=bow2doc_corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)


In [ ]:
#show topics and descriptions
df_topic_sents_keywords = show_topics_sentences(ldamodel=pickled_lda, corpus=bow2doc_corpus, texts=df['job_description'])


df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


In [ ]:
df_dominant_topic.head()

---

**Create df for topic scores for each jobtitle**

In [ ]:
topic_vecs = []
for i in range(len(bow2doc_corpus)):
    top_topics = pickled_lda.get_document_topics(bow2doc_corpus[i], minimum_probability=0.0)
    #i in range(amount of topics)
    topic_vec = [top_topics[i][1] for i in range(9)]
    topic_vecs.append(topic_vec)

In [ ]:
df_topic_vecs = pd.DataFrame(topic_vecs)
df_topic_vecs.head(10)

In [ ]:
#name columns for df

col_names=['Computer Network', 'Web Dev', 'Security', 'Analyst', 
           'Leadership', 'Database Admin', 'Cloud Computing', 'Computer Support', 'Software/App Dev']


df_topic_vecs.columns = col_names
df_topic_vecs.head()

---

---

# Nearest Neighbors

In [ ]:
#next step merge with original df of job titles and job descriptions
#pickle the merged df

In [ ]:
df_final = pd.merge(df, df_topic_vecs,left_index=True, right_index=True)

In [ ]:
#pickle.dump(df_final, open('pickled_models/df_final.pkl', 'wb'))
pickled_df_final = pickle.load(open('pickled_models/df_final.pkl', 'rb'))
pickled_df_final.head()

In [ ]:
topics = pickled_df_final.drop(['job_description', 'job_title'], axis=1)
jobs = pickled_df_final['job_title']

In [ ]:
nearest_neighbor = NearestNeighbors(n_neighbors=50)
nearest_neighbor.fit(topics)

---

**Make Recommendations**

In [ ]:
collect_score_and_recommend(nearest_neighbor, jobs)

In [36]:
#work in progress
def collect_feedback():
    """Collect user feedback and store it"""
    user_input = input("""how'd you like your recommendations? bad, okay, or good""")
    
    user_feedback = {}
    
    if user_input=='bad':
    
    elif user_input=='okay':
    
    elif user_input=='good':


SyntaxError: invalid syntax (<ipython-input-36-1f4069552664>, line 6)

In [ ]:
###### next-steps
#1.clean job-titles!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#2.recommendations return unique list of top 10
#3.seperate functions (textcleaning.py, recommend.py)
#4.create function to record feedback and store in mongodb atlas
#5.flask, level up some links from your recommendations to jobs
